In [17]:
# 主要モジュール
import torch
import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt

# 学習用のサンプルデータ
from sklearn import datasets

# データを整理する
from torch.utils.data import DataLoader

# 全結合層と活性化関数
from torch import nn
from torch.nn import functional as F

# 損失関数と最適化関数
from torch import optim
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"run with {device}")

run with cuda


In [4]:
df = pd.read_csv('../database/normaraze_stock_price/dataset_20231012.csv')

In [5]:
df

,name,2021-10-12,2021-10-13,2021-10-14,2021-10-15,2021-10-18,2021-10-19,2021-10-20,2021-10-21,2021-10-22,...,2023-09-28,2023-09-29,2023-10-02,2023-10-03,2023-10-04,2023-10-05,2023-10-06,2023-10-09,2023-10-10,2023-10-11
0,AADI,0.958538,0.950808,0.940970,0.956079,0.921644,0.936753,0.975755,0.943429,0.951862,...,0.174631,0.170063,0.151089,0.141251,0.147576,0.153549,0.157414,0.149332,0.161630,0.153900
1,AAL,0.911500,0.880952,0.882300,0.897574,0.891285,0.877359,0.876909,0.893531,0.860287,...,0.580413,0.575472,0.572776,0.552111,0.571878,0.577269,0.573226,0.549865,0.550764,0.560647
2,AAME,0.971175,0.955654,0.975610,0.960089,0.942350,0.920177,0.942350,0.929047,0.906874,...,0.414846,0.435137,0.430628,0.423864,0.412591,0.412591,0.408082,0.394554,0.408082,0.410336
3,AAOI,0.436047,0.465116,0.474160,0.466408,0.462532,0.479328,0.478682,0.485788,0.484496,...,0.711886,0.708656,0.728036,0.666667,0.575581,0.589147,0.586563,0.489987,0.481912,0.503230
4,AAON,0.619085,0.621760,0.639195,0.645745,0.653586,0.659213,0.650726,0.648051,0.652387,...,0.810549,0.801668,0.803923,0.781651,0.792787,0.789545,0.807448,0.813650,0.824786,0.832539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2644,ZUMZ,0.740436,0.730551,0.730368,0.723961,0.747392,0.734944,0.738056,0.754164,0.751236,...,0.323815,0.325828,0.316859,0.312283,0.318872,0.308805,0.307889,0.307706,0.312649,0.313381
2645,ZVRA,0.948343,0.951267,0.950292,0.952242,0.916179,0.944444,0.976608,0.949318,0.983431,...,0.482943,0.469786,0.460039,0.448343,0.447368,0.447368,0.458090,0.450292,0.462963,0.464912
2646,ZYME,0.984580,0.974942,0.953739,1.000000,0.925598,0.878951,0.890517,0.900540,0.886662,...,0.240941,0.244410,0.238628,0.238242,0.239784,0.241326,0.246723,0.242097,0.248651,0.250385
2647,ZYNE,0.955814,0.958139,0.960465,0.944186,0.906977,0.934884,0.934884,0.960465,0.900000,...,0.297674,0.296512,0.295349,0.293023,0.281395,0.290698,0.297674,0.295349,0.302326,0.302326


In [6]:
from sklearn.preprocessing import LabelEncoder

# ラベルを数値に変換する
le = LabelEncoder()
df['encode_name'] = le.fit_transform(df['name'])
df.insert(1, 'encode_name', df.pop('encode_name'))

In [7]:
df

,name,encode_name,2021-10-12,2021-10-13,2021-10-14,2021-10-15,2021-10-18,2021-10-19,2021-10-20,2021-10-21,...,2023-09-28,2023-09-29,2023-10-02,2023-10-03,2023-10-04,2023-10-05,2023-10-06,2023-10-09,2023-10-10,2023-10-11
0,AADI,0,0.958538,0.950808,0.940970,0.956079,0.921644,0.936753,0.975755,0.943429,...,0.174631,0.170063,0.151089,0.141251,0.147576,0.153549,0.157414,0.149332,0.161630,0.153900
1,AAL,1,0.911500,0.880952,0.882300,0.897574,0.891285,0.877359,0.876909,0.893531,...,0.580413,0.575472,0.572776,0.552111,0.571878,0.577269,0.573226,0.549865,0.550764,0.560647
2,AAME,2,0.971175,0.955654,0.975610,0.960089,0.942350,0.920177,0.942350,0.929047,...,0.414846,0.435137,0.430628,0.423864,0.412591,0.412591,0.408082,0.394554,0.408082,0.410336
3,AAOI,3,0.436047,0.465116,0.474160,0.466408,0.462532,0.479328,0.478682,0.485788,...,0.711886,0.708656,0.728036,0.666667,0.575581,0.589147,0.586563,0.489987,0.481912,0.503230
4,AAON,4,0.619085,0.621760,0.639195,0.645745,0.653586,0.659213,0.650726,0.648051,...,0.810549,0.801668,0.803923,0.781651,0.792787,0.789545,0.807448,0.813650,0.824786,0.832539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2644,ZUMZ,2643,0.740436,0.730551,0.730368,0.723961,0.747392,0.734944,0.738056,0.754164,...,0.323815,0.325828,0.316859,0.312283,0.318872,0.308805,0.307889,0.307706,0.312649,0.313381
2645,ZVRA,2644,0.948343,0.951267,0.950292,0.952242,0.916179,0.944444,0.976608,0.949318,...,0.482943,0.469786,0.460039,0.448343,0.447368,0.447368,0.458090,0.450292,0.462963,0.464912
2646,ZYME,2645,0.984580,0.974942,0.953739,1.000000,0.925598,0.878951,0.890517,0.900540,...,0.240941,0.244410,0.238628,0.238242,0.239784,0.241326,0.246723,0.242097,0.248651,0.250385
2647,ZYNE,2646,0.955814,0.958139,0.960465,0.944186,0.906977,0.934884,0.934884,0.960465,...,0.297674,0.296512,0.295349,0.293023,0.281395,0.290698,0.297674,0.295349,0.302326,0.302326


In [8]:
x = df.iloc[:, 2:]
i_split = 0
# 日付の特定
for i in range(len(x.columns)):
    date1 = '2023-01-01'

    # 日付をdatetimeオブジェクトに変換する
    date1_obj = datetime.strptime(x.columns[i], '%Y-%m-%d')
    date2_obj = datetime.strptime(date1, '%Y-%m-%d')

    if date1_obj > date2_obj:
        print(x.columns[i])
        print(i)
        i_split = i
        break

2023-01-03
308


In [9]:
x = df.iloc[:, i_split-60:]
label = df['encode_name']

In [10]:
x

,2022-10-04,2022-10-05,2022-10-06,2022-10-07,2022-10-10,2022-10-11,2022-10-12,2022-10-13,2022-10-14,2022-10-17,...,2023-09-28,2023-09-29,2023-10-02,2023-10-03,2023-10-04,2023-10-05,2023-10-06,2023-10-09,2023-10-10,2023-10-11
0,0.509487,0.503162,0.502460,0.478215,0.475404,0.474350,0.454322,0.452565,0.443078,0.444835,...,0.174631,0.170063,0.151089,0.141251,0.147576,0.153549,0.157414,0.149332,0.161630,0.153900
1,0.581761,0.578167,0.571878,0.547170,0.541330,0.550764,0.570530,0.586253,0.588949,0.592992,...,0.580413,0.575472,0.572776,0.552111,0.571878,0.577269,0.573226,0.549865,0.550764,0.560647
2,0.624273,0.624273,0.630961,0.613125,0.619813,0.615354,0.613125,0.622043,0.617584,0.601977,...,0.414846,0.435137,0.430628,0.423864,0.412591,0.412591,0.408082,0.394554,0.408082,0.410336
3,0.189922,0.194444,0.189276,0.179587,0.172481,0.169897,0.169897,0.176357,0.177003,0.180879,...,0.711886,0.708656,0.728036,0.666667,0.575581,0.589147,0.586563,0.489987,0.481912,0.503230
4,0.524005,0.524471,0.514233,0.494594,0.500737,0.511347,0.500085,0.513581,0.504646,0.517490,...,0.810549,0.801668,0.803923,0.781651,0.792787,0.789545,0.807448,0.813650,0.824786,0.832539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2644,0.408384,0.403258,0.406187,0.395021,0.389163,0.383855,0.387516,0.395936,0.387150,0.398499,...,0.323815,0.325828,0.316859,0.312283,0.318872,0.308805,0.307889,0.307706,0.312649,0.313381
2645,0.624756,0.588694,0.594542,0.565302,0.569201,0.571150,0.558480,0.583821,0.544834,0.547758,...,0.482943,0.469786,0.460039,0.448343,0.447368,0.447368,0.458090,0.450292,0.462963,0.464912
2646,0.235544,0.235929,0.238628,0.227448,0.230146,0.206245,0.195066,0.188512,0.212799,0.221665,...,0.240941,0.244410,0.238628,0.238242,0.239784,0.241326,0.246723,0.242097,0.248651,0.250385
2647,0.176744,0.169767,0.191860,0.172326,0.162791,0.156279,0.158372,0.154186,0.159535,0.165581,...,0.297674,0.296512,0.295349,0.293023,0.281395,0.290698,0.297674,0.295349,0.302326,0.302326


In [11]:
num_cols = len(x.columns)
grouped_cols = [x.iloc[:, i:i+59] for i in range(0, num_cols) if len(x.iloc[:, i:i+59].columns) == 59]
# 各グループを表示
grouped_cols2 = []
for group in grouped_cols:
    df_combined = pd.concat([label, group], axis=1, ignore_index=False)
    grouped_cols2.append(df_combined)
    print(df_combined.shape)

(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)
(2649, 60)

In [12]:
grouped_cols = grouped_cols2

In [13]:
grouped_cols[1]

,encode_name,2022-10-05,2022-10-06,2022-10-07,2022-10-10,2022-10-11,2022-10-12,2022-10-13,2022-10-14,2022-10-17,...,2022-12-14,2022-12-15,2022-12-16,2022-12-19,2022-12-20,2022-12-21,2022-12-22,2022-12-23,2022-12-27,2022-12-28
0,0,0.503162,0.502460,0.478215,0.475404,0.474350,0.454322,0.452565,0.443078,0.444835,...,0.460998,0.446240,0.435699,0.424455,0.432186,0.454322,0.443429,0.439916,0.421293,0.422699
1,1,0.578167,0.571878,0.547170,0.541330,0.550764,0.570530,0.586253,0.588949,0.592992,...,0.599730,0.580863,0.575472,0.560647,0.562893,0.585355,0.564241,0.570979,0.562893,0.553459
2,2,0.624273,0.630961,0.613125,0.619813,0.615354,0.613125,0.622043,0.617584,0.601977,...,0.537320,0.537320,0.577452,0.552927,0.546239,0.539550,0.557386,0.539550,0.535091,0.519484
3,3,0.194444,0.189276,0.179587,0.172481,0.169897,0.169897,0.176357,0.177003,0.180879,...,0.137597,0.136951,0.138889,0.137597,0.142765,0.137597,0.129199,0.125969,0.118217,0.112403
4,4,0.524471,0.514233,0.494594,0.500737,0.511347,0.500085,0.513581,0.504646,0.517490,...,0.745863,0.731838,0.713792,0.710894,0.705097,0.720898,0.716410,0.709304,0.719028,0.706873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2644,2643,0.403258,0.406187,0.395021,0.389163,0.383855,0.387516,0.395936,0.387150,0.398499,...,0.411312,0.383489,0.371408,0.352004,0.355299,0.369760,0.371591,0.384587,0.387150,0.388248
2645,2644,0.588694,0.594542,0.565302,0.569201,0.571150,0.558480,0.583821,0.544834,0.547758,...,0.448343,0.438596,0.439571,0.433723,0.435672,0.445419,0.427875,0.418129,0.401559,0.412281
2646,2645,0.235929,0.238628,0.227448,0.230146,0.206245,0.195066,0.188512,0.212799,0.221665,...,0.247494,0.247880,0.249036,0.261758,0.272938,0.337317,0.338859,0.309946,0.288743,0.288743
2647,2646,0.169767,0.191860,0.172326,0.162791,0.156279,0.158372,0.154186,0.159535,0.165581,...,0.139535,0.135116,0.146512,0.146512,0.132558,0.135116,0.125581,0.125581,0.116279,0.120000


In [14]:
grouped_cols[1].columns[-1]

'2022-12-28'

In [15]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)

        # 損失関数と最適化関数
        self.criterion = nn.CrossEntropyLoss()
        self.criterion_2 = nn.HingeEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x, hidden0=None):        
        # Forward pass
        out, (hidden, cell) = self.lstm(x, hidden0)
        # Index hidden state of last time step
        out = self.fc(out)
        return out

In [25]:
input_size = 60
hidden_size = 4
num_layers = 4
num_classes = 2

In [23]:
# ファイルパスを指定
file_path = '../predict_model/weights/'

# ファイル名を取得
file_names = os.listdir(file_path)

# CSVファイルのみを取得
weights = [f for f in file_names if f.endswith('.pth')]

In [24]:
weights

['model_weight_train_loss_0.15059_test_loss_1.82536_epoch_463.pth',
 'model_weight_train_loss_0.15065_test_loss_1.89213_epoch_487.pth',
 'model_weight_train_loss_0.15111_test_loss_0.74882_epoch_450.pth',
 'model_weight_train_loss_0.15502_test_loss_1.85502_epoch_477.pth',
 'model_weight_train_loss_0.15602_test_loss_1.30326_epoch_449.pth',
 'model_weight_train_loss_0.15620_test_loss_0.85109_epoch_493.pth',
 'model_weight_train_loss_0.15633_test_loss_0.76947_epoch_438.pth',
 'model_weight_train_loss_0.15643_test_loss_0.55642_epoch_439.pth',
 'model_weight_train_loss_0.15956_test_loss_1.60592_epoch_490.pth',
 'model_weight_train_loss_0.16052_test_loss_1.27359_epoch_469.pth',
 'model_weight_train_loss_0.16144_test_loss_1.89206_epoch_399.pth',
 'model_weight_train_loss_0.16283_test_loss_0.66891_epoch_405.pth',
 'model_weight_train_loss_0.16303_test_loss_0.83294_epoch_474.pth',
 'model_weight_train_loss_0.16432_test_loss_0.25740_epoch_470.pth',
 'model_weight_train_loss_0.16617_test_loss_1.25

In [225]:
grouped_cols[0].columns[-1]

'2022-12-27'

In [30]:
for weight in weights:

    # Create the model
    model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)
    model.load_state_dict(torch.load(f'../predict_model/weights/{weight}', map_location="cpu"))

    directory = f'./al_predicts/{weight}'
    if not os.path.exists(directory):
        os.mkdir(directory)

    # 各グループを表示
    for number, group in enumerate(grouped_cols):
        x = group
        # 2.DataFrameをnarrayに変換
        x = x.to_numpy().astype("float32")

        # 3.ndarrayをTensorに変換
        x = torch.from_numpy(x).to(device)

        # 4.TensorからDatasetを作成
        dataset = torch.utils.data.TensorDataset(x)

        # 6.DataLoaderに変換
        batch_size = 20
        test_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)

        # モデルを評価モードにする
        model.eval()

        # 正しい予測数、全体のデータ数を数えるカウンターの0初期化
        total_correct = 0

        name_list = []
        index_y_list = []
        ans_list = []
        val_list = []

        loss_mean = 0
        m = nn.Softmax(dim=2)

        for j, x in enumerate(test_loader):
            x = torch.stack(x)  # リストをテンソルに変換
            y = model(x)  # 順伝播（=予測）
            # print(m(y))
            
            # ミニバッチごとの正答率と損失を求める
            _, index_y = torch.max(y, axis=1)  # 最も確率が高いと予測したindex
            # print(index_y, index_t)
            res = m(y)
            val, index_y = torch.max(m(y)[0], axis=1)
            # print(val.tolist(), index_y.tolist())
            for j, (i, ii) in enumerate(zip(val.tolist(), index_y.tolist())):
                if ii == 1:
                    name_list.append(df['name'][j])
                    index_y_list.append(ii)
                    val_list.append(i)

        # リストをDataFrameに変換
        result_df = pd.DataFrame({
            'name': name_list,
            'ai_predict': index_y_list,
            'ai_confidence': val_list
        })

        result_df = result_df.sort_values(by='ai_confidence', ascending=False)

        """
        予測するのはcsvの最後の日付の次の日、ただ、その次の日は休日の可能性があるので、
        次のCSVの日付を予測日として保存する
        最後のcsvの場合は、最後の日付の次の日を予測日として保存する
        (このケースはエラーになる可能性があるので注意)
        """
        try:
            date_str = grouped_cols[number+1].columns[-1]
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
            new_date_str = date_obj.strftime('%Y-%m-%d')

        except Exception as e:
            date_str = group.columns[-1]
            date_obj = datetime.strptime(date_str, '%Y-%m-%d')
            new_date_obj = date_obj + timedelta(days=1)
            new_date_str = new_date_obj.strftime('%Y-%m-%d')

        result_df.to_csv(f'./al_predicts/{weight}/{new_date_str}.csv')

In [197]:
torch.max(m(y)[0], axis=1)

torch.return_types.max(
values=tensor([0.9630, 0.6251, 0.6053,  ..., 0.6916, 0.6741, 0.5742], device='cuda:0',
       grad_fn=<MaxBackward0>),
indices=tensor([1, 1, 0,  ..., 1, 1, 1], device='cuda:0'))

In [198]:
result_df

,name,ai_predict,ai_confidence
29,AEI,1,0.988075
28,ADTX,1,0.987467
19,ACRX,1,0.984830
37,AGFY,1,0.982886
22,ADD,1,0.982705
...,...,...,...
1171,SGH,1,0.501737
1393,VC,1,0.501423
1330,TOMZ,1,0.501298
570,HLIT,1,0.500268


In [26]:
df['0'][0]

'AADI'

In [5]:
df.iloc[:, 2:]

,2,3,4,5,6,7,8,9,10,11,...,51,52,53,54,55,56,57,58,59,60
0,0.944132,0.912860,0.946943,0.958538,0.950808,0.940970,0.956079,0.921644,0.936753,0.975755,...,0.892481,0.904076,0.907238,0.880534,0.873507,0.908292,0.864020,0.846451,0.837316,0.873858
1,0.906110,0.901617,0.904313,0.911500,0.880952,0.882300,0.897574,0.891285,0.877359,0.876909,...,0.742138,0.761456,0.777179,0.814016,0.820305,0.820305,0.816262,0.832884,0.810871,0.811770
2,0.960089,0.995565,0.997783,0.971175,0.955654,0.975610,0.960089,0.942350,0.920177,0.942350,...,0.758315,0.789357,0.691796,0.691796,0.687361,0.711752,0.678492,0.694013,0.638581,0.616408
3,0.445736,0.438631,0.439922,0.436047,0.465116,0.474160,0.466408,0.462532,0.479328,0.478682,...,0.351421,0.366279,0.347545,0.353359,0.352713,0.364341,0.361111,0.354005,0.342377,0.345607
4,0.644453,0.636428,0.626557,0.619085,0.621760,0.639195,0.645745,0.653586,0.659213,0.650726,...,0.716707,0.721151,0.713096,0.721336,0.727724,0.727447,0.738927,0.734668,0.738001,0.733002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,0.616279,0.688631,0.590439,0.574289,0.495478,0.560078,0.524548,0.510982,0.550388,0.544574,...,0.635659,0.601421,0.618863,0.606589,0.629845,0.664083,0.711886,0.708656,0.728036,0.666667
3532,0.910362,0.914211,0.933452,0.974470,0.984607,0.971654,0.960860,0.948376,0.956355,0.949597,...,0.862988,0.798990,0.802796,0.804769,0.782779,0.801668,0.810549,0.801668,0.803923,0.781651
3533,0.965996,0.969916,0.970680,0.987478,0.986154,0.993128,0.983100,0.977043,0.981166,0.985594,...,0.894513,0.886561,0.890944,0.897520,0.876519,0.868720,0.870046,0.872696,0.885643,0.878762
3534,0.669834,0.687958,0.675013,0.692581,0.726424,0.747137,0.735301,0.735301,0.755459,0.751576,...,0.705712,0.691842,0.687588,0.699794,0.688698,0.688513,0.698725,0.712837,0.701325,0.686656


In [6]:
# 1.DataFrameを説明変数と目的変数に分ける
x = df.iloc[:, 2:]
t = pd.get_dummies(df['1'])


In [7]:
x


,2,3,4,5,6,7,8,9,10,11,...,51,52,53,54,55,56,57,58,59,60
0,0.944132,0.912860,0.946943,0.958538,0.950808,0.940970,0.956079,0.921644,0.936753,0.975755,...,0.892481,0.904076,0.907238,0.880534,0.873507,0.908292,0.864020,0.846451,0.837316,0.873858
1,0.906110,0.901617,0.904313,0.911500,0.880952,0.882300,0.897574,0.891285,0.877359,0.876909,...,0.742138,0.761456,0.777179,0.814016,0.820305,0.820305,0.816262,0.832884,0.810871,0.811770
2,0.960089,0.995565,0.997783,0.971175,0.955654,0.975610,0.960089,0.942350,0.920177,0.942350,...,0.758315,0.789357,0.691796,0.691796,0.687361,0.711752,0.678492,0.694013,0.638581,0.616408
3,0.445736,0.438631,0.439922,0.436047,0.465116,0.474160,0.466408,0.462532,0.479328,0.478682,...,0.351421,0.366279,0.347545,0.353359,0.352713,0.364341,0.361111,0.354005,0.342377,0.345607
4,0.644453,0.636428,0.626557,0.619085,0.621760,0.639195,0.645745,0.653586,0.659213,0.650726,...,0.716707,0.721151,0.713096,0.721336,0.727724,0.727447,0.738927,0.734668,0.738001,0.733002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,0.616279,0.688631,0.590439,0.574289,0.495478,0.560078,0.524548,0.510982,0.550388,0.544574,...,0.635659,0.601421,0.618863,0.606589,0.629845,0.664083,0.711886,0.708656,0.728036,0.666667
3532,0.910362,0.914211,0.933452,0.974470,0.984607,0.971654,0.960860,0.948376,0.956355,0.949597,...,0.862988,0.798990,0.802796,0.804769,0.782779,0.801668,0.810549,0.801668,0.803923,0.781651
3533,0.965996,0.969916,0.970680,0.987478,0.986154,0.993128,0.983100,0.977043,0.981166,0.985594,...,0.894513,0.886561,0.890944,0.897520,0.876519,0.868720,0.870046,0.872696,0.885643,0.878762
3534,0.669834,0.687958,0.675013,0.692581,0.726424,0.747137,0.735301,0.735301,0.755459,0.751576,...,0.705712,0.691842,0.687588,0.699794,0.688698,0.688513,0.698725,0.712837,0.701325,0.686656


In [8]:
t

,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
3531,1,0
3532,0,1
3533,1,0
3534,1,0


In [9]:
# 2.DataFrameをnarrayに変換
x = x.to_numpy().astype("float32")

# 3.ndarrayをTensorに変換
x = torch.from_numpy(x).to(device)

# 4.TensorからDatasetを作成
dataset = torch.utils.data.TensorDataset(x)

# 6.DataLoaderに変換
batch_size = 20
test_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)

In [94]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()

        self.rnn = nn.LSTM(input_size = 64,
                            hidden_size = 100,
                            batch_first = True)
        self.output_layer = nn.Linear(20, 2)

        # 損失関数と最適化関数
        self.criterion = nn.CrossEntropyLoss()
        self.criterion_2 = nn.HingeEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, inputs, hidden0=None):
        output, (hidden, cell) = self.rnn(inputs, hidden0) #LSTM層
        output = self.output_layer(output[:, -1]) 
        return output

In [95]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)

        # 損失関数と最適化関数
        self.criterion = nn.CrossEntropyLoss()
        self.criterion_2 = nn.HingeEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x, hidden0=None):        
        # Forward pass
        out, (hidden, cell) = self.lstm(x, hidden0)
        # Index hidden state of last time step
        out = self.fc(out)
        return out

In [99]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # 全結合層を6つに増やす
        self.fc1 = nn.Linear(64, 32)
        self.dropout1 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc2 = nn.Linear(32, 64)
        self.dropout2 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc3 = nn.Linear(64, 64)
        self.dropout3 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc4 = nn.Linear(64, 64)
        self.dropout4 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc5 = nn.Linear(64, 32)
        self.dropout5 = nn.Dropout(0.3)  # ドロップアウトを追加
        self.fc6 = nn.Linear(32, 2)

        # 損失関数と最適化関数
        self.criterion = nn.CrossEntropyLoss()
        self.criterion_2 = nn.HingeEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)  # ドロップアウトを適用
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)  # ドロップアウトを適用
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout3(x)  # ドロップアウトを適用
        x = self.fc4(x)
        x = F.relu(x)
        x = self.dropout4(x)  # ドロップアウトを適用
        x = self.fc5(x)
        x = F.relu(x)
        x = self.dropout5(x)  # ドロップアウトを適用
        x = self.fc6(x)
        return x

In [100]:
def test(model, data_loader):
    # モデルを評価モードにする
    model.eval()

    # 正しい予測数、全体のデータ数を数えるカウンターの0初期化
    total_data_len = 0
    total_correct = 0

    loss_mean = 0

    for j, (x, t) in enumerate(data_loader):
        y = model(x)  # 順伝播（=予測）
        loss = model.criterion(y, t)  # 損失を計算
        loss_mean += loss.item()

        # ミニバッチごとの正答率と損失を求める
        _, index_y = torch.max(y, axis=1)  # 最も確率が高いと予測したindex
        _, index_t = torch.max(t, axis=1)  # 正解のindex
        for i in range(len(t)):  # データ一つずつループ,ミニバッチの中身出しきるまで
            total_data_len += 1  # 全データ数を集計
            if index_y[i] == index_t[i]:
                total_correct += 1 # 正解のデータ数を集計

    loss_mean = loss_mean / (j+1)

    return total_correct, total_data_len, loss_mean

In [101]:
input_size = 59
hidden_size = 2
num_layers = 3
num_classes = 2

# Create the model
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)
model.load_state_dict(torch.load('../predict_model/weights/model_weight3.pth', map_location="cpu"))


<All keys matched successfully>

In [17]:
# モデルを評価モードにする
model.eval()

# 正しい予測数、全体のデータ数を数えるカウンターの0初期化
total_data_len = 0
total_correct = 0

name_list = []
index_y_list = []
ans_list = []
val_list = []

loss_mean = 0
m = nn.Softmax(dim=2)

for j, x in enumerate(test_loader):
    x = torch.stack(x)  # リストをテンソルに変換
    y = model(x)  # 順伝播（=予測）
    # print(m(y))
    
    
    # ミニバッチごとの正答率と損失を求める
    _, index_y = torch.max(y, axis=1)  # 最も確率が高いと予測したindex
    # print(index_y, index_t)
    for i in range(len(t)):  # データ一つずつループ,ミニバッチの中身出しきるまで
        total_data_len += 1  # 全データ数を集計
    res = m(y)
    val, index_y = torch.max(m(y)[0], axis=1)
    print(val.tolist(), index_y.tolist())
    for j, (i, ii) in enumerate(zip(val.tolist(), index_y.tolist())):
        name_list.append(df['0'][j])
        index_y_list.append(ii)
        val_list.append(i)

[0.6188809275627136, 0.5900458097457886, 0.7202277779579163, 0.6217108964920044, 0.6510367393493652, 0.5682734847068787, 0.6661288738250732, 0.6205351948738098, 0.6994403004646301, 0.6248250603675842, 0.741827130317688, 0.6049798130989075, 0.6640167832374573, 0.5857458710670471, 0.6653698086738586, 0.5793137550354004, 0.7054375410079956, 0.5996900200843811, 0.741631805896759, 0.6484270095825195, 0.6529103517532349, 0.6232622861862183, 0.661768913269043, 0.638318657875061, 0.6749601364135742, 0.6101622581481934, 0.7435817718505859, 0.5827338695526123, 0.6395730376243591, 0.6243159174919128, 0.6500524282455444, 0.630545973777771, 0.6717182397842407, 0.6300795674324036, 0.744522750377655, 0.6389649510383606, 0.6072825789451599, 0.6068164110183716, 0.639148473739624, 0.6837903261184692, 0.6729153990745544, 0.6194990277290344, 0.7362712621688843, 0.6448964476585388, 0.6151716113090515, 0.615625262260437, 0.6318235993385315, 0.7192761301994324, 0.6588127017021179, 0.6537846922874451, 0.74664

In [43]:
# リストをDataFrameに変換
result_df = pd.DataFrame({
    'name': name_list,
    'index_y': index_y_list,
    'ai_confidence': val_list
})

In [46]:
# 7つずつに分割する
result_df_split = np.array_split(result_df, len(result_df) // interval + 1)

# 分割したデータフレームを表示する
for df in result_df_split:
    # val列で降順にソート
    sorted_df = df.sort_values(by='ai_confidence', ascending=False)
    print(sorted_df.head(1))

   name  index_y  ai_confidence
2  AAME        0       0.720228
    name  index_y  ai_confidence
10  AAME        0       0.741827
    name  index_y  ai_confidence
18  AAME        0       0.741632
    name  index_y  ai_confidence
26  AAME        0       0.743582
    name  index_y  ai_confidence
34  AAME        0       0.744523
    name  index_y  ai_confidence
42  AAME        0       0.736271
    name  index_y  ai_confidence
50  AAME        0       0.746645
    name  index_y  ai_confidence
63  ABCL        0       0.717605
    name  index_y  ai_confidence
66  AAME        0        0.73218
    name  index_y  ai_confidence
74  AAME        0       0.718493
    name  index_y  ai_confidence
82  AAME        0       0.722851
    name  index_y  ai_confidence
90  AAME        0       0.719773
    name  index_y  ai_confidence
99  AAOI        0       0.768365
     name  index_y  ai_confidence
107  AAOI        0       0.754259
     name  index_y  ai_confidence
114  AAME        0       0.732721
     nam

In [19]:
result_df

,name,index_y,val
0,AADI,0,0.618881
1,AAL,0,0.590046
2,AAME,0,0.720228
3,AAOI,0,0.621711
4,AAON,1,0.651037
...,...,...,...
3531,AAOI,1,0.828456
3532,AAON,0,0.710090
3533,AAPL,0,0.703670
3534,ABCB,0,0.554590


In [130]:
# val列で降順にソート
result_df = result_df.sort_values(by='val', ascending=False)

# DataFrameの内容を表示
print(result_df)

In [134]:
# csv_path = "pread_AI.csv"
# headers = ','.join(t.columns.to_list())
# with open(csv_path, "w", encoding="utf_8") as csv_file:
#     csv_file.write(headers + '\n')
pred_list = []
for data_raw in dataset:
    data = torch.Tensor(data_raw[0]).to(device)
    pred = model(data)
    pred = F.softmax(pred, dim=0)
    pred_list.append(pred.tolist())

    # with open(csv_path, 'a') as file:
    #     writer = csv.writer(file)
    #     writer.writerow(pred_list)